In [ ]:
!pip install -q chromadb==0.4.22
!pip install -q langchain==0.1.4
!pip install -q sentence_transformers==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 23.

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
from google.colab import files

# Prompt the user to upload a file
uploaded = files.upload()

Saving labelled_newscatcher_dataset.csv to labelled_newscatcher_dataset.csv


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
news = pd.read_csv('labelled_newscatcher_dataset.csv', sep=';')
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 500
#DOCUMENT="description"
#TOPIC="title"

#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

In [ ]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [ ]:
df_loader = DataFrameLoader(subset_news, page_content_column=DOCUMENT)

In [ ]:
df_document = df_loader.load()
# display(df_document)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:

text_splitter = CharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=10,
)
texts = text_splitter.split_documents(df_document)

In [ ]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
import chromadb
from chromadb.config import Settings

In [ ]:
db2 = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")

In [ ]:
from langchain.chains import RetrievalQA
retriever = db2.as_retriever()

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [ ]:
document_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

In [ ]:
response = document_qa.run("Which laptop shall i buy?")

In [ ]:
display(response)

"I don't know. The provided context does not contain enough information to make a recommendation on which laptop to buy."

In [ ]:
response = document_qa.run("Can I buy a Toshiba laptop?")

In [ ]:
display(response)

'No, Toshiba is no longer making laptops.'

Here I didnot find the response awesone. Insted for this type of task  "databricks/dolly-v2-3b" would give better response according to my knowledge.

With the aid of embedding techniques, the output I obtained was significantly improved. Through embedding on my dataset, I gained contextual understanding, and employing the LLM model resulted in a substantially enhanced output, delivering meaningful answers.

